In [14]:
import pandas as pd

df = pd.read_csv("socialMedia.csv")

print(df.head())
print(df.describe(include="all"))
print(df.isnull().sum())
print(df.columns)

   id_aluno  idade         país nivel_educacional  horas_uso_diario  \
0         1     18       Brasil      Ensino Médio               5.0   
1         2     22          EUA         Graduação               7.5   
2         3     20        Índia         Graduação               3.0   
3         4     25  Reino Unido     Pós-graduação               6.5   
4         5     19       Canadá      Ensino Médio               2.0   

  plataforma_favorita          estado_civil  grau_conflito  \
0           Instagram              Solteiro              2   
1              TikTok  Em um relacionamento              6   
2            WhatsApp              Solteiro              1   
3            Facebook                Casado              4   
4            Snapchat  Em um relacionamento              3   

                                     resposta_aberta  
0  Passo horas vendo reels e conversando com amigos.  
1  Meu parceiro fica irritado quando ignoro mensa...  
2  Uso principalmente para manter c

In [15]:
import spacy

# Carrega o modelo em português
nlp = spacy.load("pt_core_news_sm")

def preprocess(text):
    if not isinstance(text, str):
        return ""
    
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    print(tokens)
    return " ".join(tokens)

df["processed_text"] = df["resposta_aberta"].apply(preprocess)

#comparacao entre o originval e pos processamento com remocao de stopwords
print(df[["resposta_aberta", "processed_text"]].head())

['passo', 'hora', 'ver', 'reels', 'conversar', 'amigo']
['parceiro', 'ficar', 'irritar', 'ignorar', 'mensagem', 'causa', 'rolagem']
['uso', 'principalmente', 'manter', 'contato', 'colega', 'classe']
['hábito', 'afetar', 'sono', 'família']
['verifico', 'mensagem']
                                     resposta_aberta  \
0  Passo horas vendo reels e conversando com amigos.   
1  Meu parceiro fica irritado quando ignoro mensa...   
2  Uso principalmente para manter contato com col...   
3  É um hábito, às vezes afeta meu sono e o tempo...   
4  Não é grande coisa, só verifico as mensagens d...   

                                      processed_text  
0               passo hora ver reels conversar amigo  
1  parceiro ficar irritar ignorar mensagem causa ...  
2    uso principalmente manter contato colega classe  
3                         hábito afetar sono família  
4                                  verifico mensagem  


In [16]:
from sentence_transformers import SentenceTransformer

# Carrega um modelo pré-treinado que funciona bem com português
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Lista com os textos para gerar embeddings (exemplo: suas respostas abertas já processadas)
texts = df["processed_text"].tolist()

# Gera os embeddings
embeddings = model.encode(texts)

print(f"Gerados {len(embeddings)} embeddings com dimensão {embeddings[0].shape}")


Gerados 5 embeddings com dimensão (384,)


In [17]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# Inicializa o cliente ChromaDB (local, sem configuração extra)
client = chromadb.Client(Settings())

# Cria (ou acessa) uma coleção chamada "respostas_alunos"
collection = client.get_or_create_collection(name="respostas_alunos")

# Carrega o modelo para gerar embeddings
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Textos já pré-processados, por exemplo:
texts = df["processed_text"].tolist()

# Gera embeddings
embeddings = model.encode(texts).tolist()  # converte para lista para Chroma

# IDs para cada vetor (pode ser o id do aluno)
ids = df["id_aluno"].astype(str).tolist()

# Adiciona os vetores na coleção
collection.add(
    ids=ids,
    documents=texts,   # opcional, mas bom para referência no search
    embeddings=embeddings
)

print("Embeddings armazenados com sucesso no ChromaDB!")


Embeddings armazenados com sucesso no ChromaDB!


In [18]:
query = "Estou cansado de ficar no celular o tempo todo"

query_embedding = model.encode([query]).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
)

print(results)


{'ids': [['1', '3', '2']], 'embeddings': None, 'documents': [['passo hora ver reels conversar amigo', 'uso principalmente manter contato colega classe', 'parceiro ficar irritar ignorar mensagem causa rolagem']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None, None]], 'distances': [[22.70426368713379, 25.72380256652832, 27.543392181396484]]}
